In [1]:
# TODO: THAT WORKS, NOW WE ONLY NEED TO FIX EVERYTHING IN THE ENTIRE PROJECT TO ADAT TO THE NEW API
# TODO: WE FIRST NEED TO FINISH IMPLEMENTING THE SIM API FOR THE NEW ALGORITHM, AND ONLY AFTERWARDS ADAPRT THE REST OF THE PROJECT PARTS. 

from simulator import Simulator

sim = Simulator(world_file="data/mug/world.xml", resolution=(250, 300), fov=35)

sim.set_object_location((10, 10, 10))
sim.align_camera(cam_dist=1)

In [2]:
from scipy.spatial.transform import Rotation
import numpy as np


def aligned_orientation(theta: float, phi: float, psi: float = 0.0) -> Rotation:
    """
    Search over orientations with the last angle as a rotation around [0,1,0].

    Parameters:
    - theta: Rotation around the x-axis in radians. Range: [0, pi].
    - phi: Rotation around the z-axis in radians. Range: [0, 2*pi].
    - psi: Rotation around the [0,1,0] axis (yaw) in radians. Range: [0, 2*pi].

    Returns:
    - rot: A scipy Rotation object representing the orientation.
    """
    rx = Rotation.from_euler("x", theta)
    rz = Rotation.from_euler("z", phi)
    ry = Rotation.from_euler("y", psi)
    rot = rx * rz * ry
    return rot

In [3]:
import loss_funcs

import numpy as np
from skimage.registration import phase_cross_correlation
from skimage import transform
from skimage.color import rgb2gray


ref_angles = np.random.uniform(low=np.array([0, 0, 0]), high=np.array([np.pi, 2 * np.pi, 2 * np.pi]))

ref_orient = aligned_orientation(ref_angles[0], ref_angles[1], ref_angles[2])

sim.set_object_orientation(ref_orient.as_euler("xyz"))

ref_img = sim.render()

w, h = ref_img.shape[1], ref_img.shape[0]

radius = np.sqrt(w**2 + h**2) / 2

polar_shape = (360, int(np.ceil(radius)))

ref_polar = transform.warp_polar(rgb2gray(ref_img), radius=radius, output_shape=polar_shape)

loss_func = loss_funcs.IOU()

def obj_func_1(angle: np.ndarray) -> float:

    theta, phi = angle[0], angle[1]

    orient = aligned_orientation(theta, phi)

    sim.set_object_orientation(orient.as_euler("xyz"))

    eval_img = sim.render()

    eval_polar = transform.warp_polar(rgb2gray(eval_img), radius=radius, output_shape=polar_shape)

    ref_fft = np.fft.fft(ref_polar, axis=0)
    eval_fft = np.fft.fft(eval_polar, axis=0)
    shifts, _, _ = phase_cross_correlation(ref_fft, eval_fft, space="fourier")
    
    #shifts, _, _ = phase_cross_correlation(ref_polar, eval_polar, space="real")

    pred_deg = shifts[0] / eval_polar.shape[0] * 360
    pred_deg = pred_deg % 360
    pred_rad = pred_deg / 180 * np.pi

    aligned_orient = aligned_orientation(theta, phi, -pred_rad)

    sim.set_object_orientation(aligned_orient.as_euler("xyz"))

    test_img = sim.render()

    return loss_func(ref_img, test_img)

In [ ]:
from algs.mealpy_algorithm import MealTermination, MealRunConfig
import mealpy
from mealpy.utils.agent import Agent
from algs.algorithm import RunHistory


bounds = mealpy.FloatVar(lb=np.asanyarray([0, 0]), ub=np.asanyarray([np.pi, 2*np.pi]))

problem = mealpy.Problem(
    obj_func=obj_func_1,
    bounds=bounds,
    minmax="min",
    log_to="console",
    name="orient_detection_position",
    save_population=False,
)

termination = MealTermination(10000, 100 * (2 / 3))

optim = mealpy.PSO.OriginalPSO(c1=1, c2=2.05, w=0.2)

best: Agent = optim.solve(
    problem=problem,
    termination=termination,
    starting_solutions=None,
    seed=None,
)

run_hist = RunHistory(optim.history.list_epoch_time, optim.history.list_global_best_fit)

In [5]:
phase1_sol = best.solution

theta, phi = phase1_sol[0], phase1_sol[1]

def obj_func_2(angle: np.ndarray) -> float:

    orient = aligned_orientation(theta, phi, angle[0])

    sim.set_object_orientation(orient.as_euler("xyz"))

    test_img = sim.render()

    return loss_func(ref_img, test_img)

In [ ]:
from algs.mealpy_algorithm import MealTermination, MealRunConfig
import mealpy
from mealpy.utils.agent import Agent
from algs.algorithm import RunHistory


bounds = mealpy.FloatVar(lb=np.asanyarray([0]), ub=np.asanyarray([2 * np.pi]))

problem = mealpy.Problem(
    obj_func=obj_func_2,
    bounds=bounds,
    minmax="min",
    log_to="console",
    name="orient_detection_rotation",
    save_population=False,
)

termination = MealTermination(10000, 100 * (1 / 3))

optim = mealpy.PSO.OriginalPSO(c1=1, c2=2.05, w=0.2)

best: Agent = optim.solve(
    problem=problem,
    termination=termination,
    starting_solutions=None,
    seed=None,
)

run_hist = RunHistory(optim.history.list_epoch_time, optim.history.list_global_best_fit)

In [7]:
import loss_funcs

sim.set_object_orientation(ref_orient.as_euler("xyz"))

ref_img = sim.render()

loss_func = loss_funcs.IOU()

def obj_func_orig(angle: np.ndarray) -> float:

    sim.set_object_orientation(angle)

    test_img = sim.render()

    return loss_func(ref_img, test_img)

In [ ]:
from algs.mealpy_algorithm import MealTermination, MealRunConfig
import mealpy
from mealpy.utils.agent import Agent
from algs.algorithm import RunHistory


bounds = mealpy.FloatVar(lb=np.asanyarray([0, 0, 0]), ub=np.asanyarray([np.pi, 2*np.pi, 2*np.pi]))

problem = mealpy.Problem(
    obj_func=obj_func_orig,
    bounds=bounds,
    minmax="min",
    log_to="console",
    name="orient_detection_position",
    save_population=False,
)

termination = MealTermination(10000, 100)

optim = mealpy.PSO.OriginalPSO(c1=1, c2=2.05, w=0.2)

best: Agent = optim.solve(
    problem=problem,
    termination=termination,
    starting_solutions=None,
    seed=None,
)

run_hist = RunHistory(optim.history.list_epoch_time, optim.history.list_global_best_fit)